# MNIST Single Layer Network with TensorBoard

Note: This notebook is desinged to run with Python3 and GPU runtime.

![Python 3 and CPU runtime](https://raw.githubusercontent.com/enakai00/colab_tfbook/master/docs/imgs/runtime_gpu.png)

In [1]:
%%bash
tf_version='2.0.0rc0'
if pip freeze | grep -q "^tensorflow==${tf_version}$"; then
  echo "tensorflow==${tf_version} is already installed. No actions are required."
else
  echo "Installing tensorflow==${tf_version}. Don't forget to restart the runtime."
  pip install tensorflow==${tf_version}
fi

tensorflow==2.0.0rc0 is already installed. No actions are required.


####[MST-01]
Import modules and set random seeds.

In [0]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import shutil

import tensorflow as tf
from tensorflow.keras import layers, models, initializers, callbacks
from tensorflow.keras.datasets import mnist

np.random.seed(20190228)
tf.random.set_seed(20190228)

####[MST-02]
Download the MNIST dataset and store into NumPy arrays.

In [0]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape(
                  (len(train_images), 784)).astype('float32') / 255
test_images = test_images.reshape(
                  (len(test_images), 784)).astype('float32') / 255
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

####[MST-03]
Define a model with a single hidden layer.

In [4]:
model = models.Sequential()
model.add(layers.Dense(1024, activation='relu', input_shape=(28*28,),
                       kernel_initializer=initializers.TruncatedNormal(),
                       name='hidden'))
model.add(layers.Dense(10, activation='softmax', name='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 1024)              803840    
_________________________________________________________________
softmax (Dense)              (None, 10)                10250     
Total params: 814,090
Trainable params: 814,090
Non-trainable params: 0
_________________________________________________________________


####[MST-04]
Compile the model using the Adam optimizer, and Cross entroy as a loss function.

In [0]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['acc'])

####[MST-05]
Train the model with the callbacks option to store training logs.

In [6]:
log_dir = '/tmp/log'
shutil.rmtree(log_dir, ignore_errors=True)
tensorboard_callback = callbacks.TensorBoard(log_dir=log_dir,
                                             histogram_freq=1)
history = model.fit(train_images, train_labels,
                    validation_data=(test_images, test_labels),
                    batch_size=128, epochs=10,
                    callbacks=[tensorboard_callback])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 7s 114us/sample - loss: 0.2311 - acc: 0.9339 - val_loss: 0.1189 - val_acc: 0.9643
Epoch 2/10
60000/60000 [==============================] - 6s 104us/sample - loss: 0.0909 - acc: 0.9734 - val_loss: 0.0828 - val_acc: 0.9743
Epoch 3/10
60000/60000 [==============================] - 6s 102us/sample - loss: 0.0558 - acc: 0.9835 - val_loss: 0.0669 - val_acc: 0.9788
Epoch 4/10
60000/60000 [==============================] - 6s 105us/sample - loss: 0.0390 - acc: 0.9887 - val_loss: 0.0619 - val_acc: 0.9801
Epoch 5/10
60000/60000 [==============================] - 6s 103us/sample - loss: 0.0254 - acc: 0.9925 - val_loss: 0.0589 - val_acc: 0.9821
Epoch 6/10
60000/60000 [==============================] - 6s 105us/sample - loss: 0.0186 - acc: 0.9949 - val_loss: 0.0605 - val_acc: 0.9823
Epoch 7/10
60000/60000 [==============================] - 6s 105us/sample - loss: 0.0146 - acc: 0.9957 - val_l

####[MST-06]
Install ngrok to run TensorBoard on Colaboratory.

In [7]:
!curl -OL https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12.9M  100 12.9M    0     0  43.5M      0 --:--:-- --:--:-- --:--:-- 43.5M
Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


####[MST-07]
Start TensorBoard and prepare the connection URL.

In [8]:
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(log_dir)
)
get_ipython().system_raw('./ngrok http 6006 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://9d60b02e.ngrok.io
